In [90]:
from pathlib import Path
from espnet2.bin.enh_variable_number_speakers_inference import SeparateSpeech

epoch_number = 174
model_dir = Path("./exp/enh_enh_dptnet_eda4thlayer_2-5mix_1bce_raw")
innference_module = SeparateSpeech(model_dir/"config.yaml", model_dir/f"{epoch_number}epoch.pth")


Task is enh
Number of speakers 5


In [91]:
from espnet2.fileio.read_text import read_2columns_text
data_path = Path("./dump/raw/tt_min_8k")
mix_scp = read_2columns_text(data_path/"wav.scp")
ref_scps = []
for i in range(1,6,1):
    ref_scps.append(read_2columns_text(data_path/f"spk{i}.scp"))

In [92]:
est_num_spk = model_dir / f"enhanced_tt_min_8k_{epoch_number}epoch" / "enh" / "without_true_numspk" / "scoring" / "Est_num_spk"
est_num_spk = read_2columns_text(est_num_spk)
mis_estimation = {2: [], 3: [], 4: [], 5: []}
for id, nspk in est_num_spk.items():
    if id[0] != nspk:
        mis_estimation[int(id[0])].append((nspk, id))
for nspk in mis_estimation:
    print(nspk, len(mis_estimation[nspk]))

2 3
3 23
4 94
5 211


In [99]:
from espnet2.torch_utils.set_all_random_seed import set_all_random_seed
import fast_bss_eval
import soundfile as sf
import torch

# [4][1] [4][3] is a good example
org_est_nspk, example_id = mis_estimation[4][15]
mix = torch.from_numpy(sf.read(mix_scp[example_id], dtype="float32")[0])
ref = []
for i in range(5):
    if ref_scps[i][example_id] != "dummy":
        ref.append(torch.from_numpy(sf.read(ref_scps[i][example_id], dtype="float32")[0]))
ref = torch.stack(ref, dim=0)

print(f"Original estimated number is : {org_est_nspk}\n")
for seed in range(10):
    set_all_random_seed(seed)
    est = innference_module(mix[None])
    est = torch.cat(est, dim=0)
    print(f"Seed: {seed}, Estimated number is: {est.shape[0]}")
    sisdr, perm = fast_bss_eval.si_sdr(ref, est, return_perm=True)
    print(sisdr, perm)

Original estimated number is : 2

Seed: 0, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
Seed: 1, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
Seed: 2, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
Seed: 3, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
Seed: 4, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
Seed: 5, Estimated number is: 2
tensor([19.7507, 10.1679]) tensor([0, 1])
Seed: 6, Estimated number is: 2
tensor([19.7507, 10.1679]) tensor([0, 1])
Seed: 7, Estimated number is: 2
tensor([19.7507, 10.1679]) tensor([0, 1])
Seed: 8, Estimated number is: 2
tensor([19.7507, 10.1679]) tensor([0, 1])
Seed: 9, Estimated number is: 2
tensor([19.7506, 10.1679]) tensor([0, 1])
